<a href="https://colab.research.google.com/github/Ejmaldonadov/Ejmaldonadov.github.io/blob/main/Proyecto_Sistema_de_Recomendaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import random

# Configuración
usuarios = [f"U{i+1}" for i in range(50)]
contenidos = [f"C{i+1}" for i in range(10)]
datos = []

# Generar calificaciones aleatorias (pero no todos califican todo)
for usuario in usuarios:
    contenidos_calificados = random.sample(contenidos, k=random.randint(3, 7))  # Cada usuario califica entre 3 y 7 contenidos
    for contenido in contenidos_calificados:
        calificacion = random.randint(1, 5)
        datos.append({'Usuario': usuario, 'Contenido': contenido, 'Calificacion': calificacion})

# Crear DataFrame
df = pd.DataFrame(datos)

# Mostrar los primeros datos
print(df.head())

print("-----------------------")
# Crear la matriz de usuario–contenido
matriz = df.pivot_table(index='Usuario', columns='Contenido', values='Calificacion')

# Mostrar las primeras filas
print(matriz.head())

print("-----------------------")
from sklearn.metrics.pairwise import cosine_similarity

# Rellenar valores NaN con 0 para poder calcular similitud
matriz_usuarios = matriz.fillna(0)

# Calcular la similitud del coseno entre usuarios
similitud = cosine_similarity(matriz_usuarios)

# Crear DataFrame con la matriz de similitud
similitud_df = pd.DataFrame(similitud, index=matriz_usuarios.index, columns=matriz_usuarios.index)

# Mostrar una parte de la matriz
print(similitud_df.iloc[:5, :5])  # Solo una muestra

print("---------------------------")
# Escoger usuario objetivo
usuario_objetivo = 'U33'

# Obtener las calificaciones del usuario y sus similitudes
calificaciones_usuario = matriz.loc[usuario_objetivo]
similitudes = similitud_df[usuario_objetivo]

# Calcular predicciones para contenidos no calificados
predicciones = {}

for contenido in matriz.columns:
    if pd.isna(calificaciones_usuario[contenido]):
        calificaciones_contenido = matriz[contenido]

        numerador = 0
        denominador = 0

        for otro_usuario in matriz.index:
            if otro_usuario != usuario_objetivo and not pd.isna(calificaciones_contenido[otro_usuario]):
                sim = similitudes[otro_usuario]
                calif = calificaciones_contenido[otro_usuario]
                numerador += sim * calif
                denominador += sim

        if denominador != 0:
            prediccion = numerador / denominador
            predicciones[contenido] = prediccion

# Ordenar recomendaciones
recomendaciones = sorted(predicciones.items(), key=lambda x: x[1], reverse=True)

# Mostrar las recomendaciones
print(f"Recomendaciones para {usuario_objetivo}")
for contenido, score in recomendaciones:
    print(f"{contenido}: {score:.2f}")


  Usuario Contenido  Calificacion
0      U1        C6             3
1      U1        C8             4
2      U1        C7             2
3      U1        C2             3
4      U1        C4             2
-----------------------
Contenido   C1  C10   C2   C3   C4   C5   C6   C7   C8  C9
Usuario                                                   
U1         NaN  NaN  3.0  NaN  2.0  3.0  3.0  2.0  4.0 NaN
U10        5.0  NaN  5.0  4.0  NaN  5.0  2.0  3.0  2.0 NaN
U11        NaN  NaN  5.0  2.0  NaN  NaN  NaN  NaN  4.0 NaN
U12        1.0  2.0  1.0  NaN  5.0  NaN  NaN  1.0  2.0 NaN
U13        NaN  NaN  3.0  5.0  2.0  NaN  2.0  NaN  4.0 NaN
-----------------------
Usuario        U1       U10       U11       U12       U13
Usuario                                                  
U1       1.000000  0.673710  0.647098  0.536774  0.643530
U10      0.673710  1.000000  0.588120  0.272638  0.593843
U11      0.647098  0.588120  1.000000  0.322988  0.802534
U12      0.536774  0.272638  0.322988  1.0000